In [46]:
import os
import rasterio.mask

import contextily as cx
import geopandas as gpd
import matplotlib.colors as mcol
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio as rio

from pathlib import Path

In [55]:
pg_path = Path(os.getenv("PG_PATH"))
ghsl_path = Path(os.getenv("GHSL_PATH"))
data_path = Path(os.getenv("DATA_PATH"))

raster_path = ghsl_path / "POP_100_CHUNKED/2020"

In [60]:
zone_list = gpd.read_file(pg_path / "initial/metropoli/2020/Metropolis_2020.shp")["CVE_MET"].unique()

In [52]:
def calculate_pops(df: gpd.GeoDataFrame):
    pop = {}
    for path in raster_path.glob("*.tif"):
        with rio.open(path, nodata=-200) as ds:
            for i, geom in df["geometry"].items():
                try:
                    masked, _ = rio.mask.mask(ds, [geom], crop=True, nodata=-200)
                except ValueError:
                    continue
                
                masked = masked.astype(float)
                masked[masked == -200] = np.nan
                
                if i in pop:
                    pop[i] += np.nansum(masked)
                else:
                    pop[i] = np.nansum(masked)

    pop = pd.Series(pop)

    df = df.assign(ghsl=pop, difference=lambda df: df.census - df.ghsl)
    return df

In [65]:
figure_path = data_path / "figures/ghsl_inside_agebs"
df_path = data_path / "polygons/ghsl_inside_agebs"

for cve in zone_list:
    try:
        df = gpd.read_file(pg_path / f"final/zone_agebs/shaped/2020/{cve}.gpkg").to_crs("ESRI:54009").rename(columns={"POBTOT": "census"})
    except Exception:
        continue

    df = calculate_pops(df)
    df.to_file(df_path / f"{cve}.gpkg")

    norm = mcol.TwoSlopeNorm(0, df["difference"].min(), df["difference"].max())

    fig, ax = plt.subplots(figsize=(10, 10))
    df.plot(column="difference", cmap="RdBu", norm=norm, legend=True, ax=ax, ec="grey", lw=0.3)
    ax.axis("off")
    cx.add_basemap(ax=ax, crs="ESRI:54009", source=cx.providers.CartoDB.Positron)

    ax.set_title(f"{cve}\ncenso - GHSL")

    fig.savefig(figure_path / f"{cve}.jpg", bbox_inches="tight", dpi=150)
    plt.close()